In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Case Técnico – Análise de Dados (Python)
Dataset: ANP – preços de combustíveis
Requisitos: pandas, numpy, matplotlib
Autores do original em R: Natan Milanez Polly e Fernando Souza Silva

Observações:
- Este script assume que os arquivos CSV estão em ./dataset (subpastas permitidas).
- Ele padroniza nomes de arquivos fora do padrão para ca-AAAA-MM.csv.
- Lê todos os CSVs (sep=";") como string e depois faz limpeza/conversões.
- Os gráficos usam apenas matplotlib (sem seaborn).
"""

from __future__ import annotations

import os
import re
import sys
import warnings
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# Configurações gerais
# ------------------------------------------------------------
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 180)
warnings.filterwarnings("ignore", category=UserWarning)

DATASET_DIR = Path("dataset")
PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)


# ------------------------------------------------------------
# Utilidades
# ------------------------------------------------------------
def sanitize_col(name: str) -> str:
    """
    Converte nomes de colunas para um formato padrão semelhante ao gsub do R:
    - remove tokens 'de' e 'da' isolados
    - substitui ' - ' por '_'
    - troca qualquer caractere não alfanumérico por '_'
    - evita multiplos underscores e tira underscores das pontas
    """
    s = name
    # normaliza espaços
    s = re.sub(r"\s+", " ", s, flags=re.UNICODE)
    # remove ' de ' e ' da ' (tokens)
    s = re.sub(r"(?i)\bde\b", "", s)
    s = re.sub(r"(?i)\bda\b", "", s)
    # substitui separador com hífen central
    s = s.replace(" - ", "_")
    # troca qualquer coisa não [\w] por underscore
    s = re.sub(r"[^\w]+", "_", s, flags=re.UNICODE)
    # reduz múltiplos underscores
    s = re.sub(r"__+", "_", s)
    # tira underscores nas pontas
    s = s.strip("_")
    return s


def first_existing(df: pd.DataFrame, candidates: Iterable[str]) -> Optional[str]:
    """Retorna o primeiro nome de coluna existente em df dentre os candidatos."""
    for c in candidates:
        if c in df.columns:
            return c
    return None


def as_float_pt(series: pd.Series) -> pd.Series:
    """Converte série string com vírgula decimal para float, preenchendo NAs com 0."""
    if series.dtype == "float" or series.dtype == "int":
        return series.astype(float)
    return (
        series.astype(str)
        .str.replace(",", ".", regex=False)
        .str.replace(" ", "", regex=False)
        .pipe(pd.to_numeric, errors="coerce")
        .fillna(0.0)
    )


def ensure_datetime(series: pd.Series) -> pd.Series:
    """Converte série string no formato dd/mm/yyyy para datetime (dia primeiro)."""
    return pd.to_datetime(series, errors="coerce", dayfirst=True)


def show_head(df: pd.DataFrame, n: int = 5, title: str = "") -> None:
    if title:
        print("\n" + title)
    print(df.head(n).to_string(index=False))


def show_tail(df: pd.DataFrame, n: int = 5, title: str = "") -> None:
    if title:
        print("\n" + title)
    print(df.tail(n).to_string(index=False))


# ------------------------------------------------------------
# 0) Padroniza nomes de arquivos "fora do padrão"
# ------------------------------------------------------------
print("# 0) Renomeando arquivos fora do padrão para ca-AAAA-MM.csv")

pattern = re.compile(r"^(.{10,33})(\d{4})[.-](\d{2})\.csv$", flags=re.IGNORECASE)

unpatterned_files: List[Path] = []
for path in DATASET_DIR.rglob("*.csv"):
    if pattern.match(path.name):
        unpatterned_files.append(path)

if unpatterned_files:
    print("Arquivos a renomear:")
    for p in unpatterned_files:
        print(" -", p.as_posix())

    for src in unpatterned_files:
        m = pattern.match(src.name)
        assert m is not None
        year = m.group(2)
        month = m.group(3)
        dst = src.with_name(f"ca-{year}-{month}.csv")  # mantém a mesma pasta
        try:
            src.rename(dst)
        except Exception as e:
            print(f"Falha ao renomear {src} -> {dst}: {e}", file=sys.stderr)
else:
    print("Nenhum arquivo fora do padrão encontrado.")

# ------------------------------------------------------------
# 1) Importa todos os CSVs do dataset
# ------------------------------------------------------------
print("\n1) Importando CSVs do diretório ./dataset (recursivo)")
all_csvs = sorted([p for p in DATASET_DIR.rglob("*.csv")])
for p in all_csvs[:5]:
    print(" -", p.as_posix())
if not all_csvs:
    print("ATENÇÃO: Nenhum CSV encontrado em ./dataset.")
    sys.exit(0)

dfs: List[pd.DataFrame] = []
for csv_path in all_csvs:
    try:
        # read_csv2 do R -> sep=';'. Ler como string inicialmente.
        df_part = pd.read_csv(
            csv_path,
            sep=";",
            dtype=str,
            encoding="latin1",
            engine="python",
        )
        dfs.append(df_part)
    except Exception as e:
        print(f"Erro ao ler {csv_path}: {e}", file=sys.stderr)

fuel_dataframe = pd.concat(dfs, ignore_index=True)
# remove linhas completamente vazias
fuel_dataframe = fuel_dataframe.dropna(how="all")

print(f"Total de linhas após concatenação: {len(fuel_dataframe):,}")
show_head(fuel_dataframe, 5, "Pré-rename (amostra)")


# ------------------------------------------------------------
# 2) Contexto de negócio (comentários explicativos no próprio script)
# ------------------------------------------------------------
# (Conteúdo descritivo mantido como comentário no topo do arquivo)


# ------------------------------------------------------------
# 3) Solução / pipeline proposto (comentários no topo)
# ------------------------------------------------------------


# ------------------------------------------------------------
# 18) Renomeia colunas específicas e depois padroniza todas
# ------------------------------------------------------------
print("\n18) Renomeando colunas principais e padronizando todos os nomes")

# Renomeia especificamente antes de sanitizar
rename_map = {
    "Valor de Venda": "Valor_Venda",
    "Valor de Compra": "Valor_Compra",
}
fuel_dataframe = fuel_dataframe.rename(columns=rename_map)

# Padroniza todos os nomes (similar ao rename_with + gsub do R)
fuel_dataframe.columns = [sanitize_col(c) for c in fuel_dataframe.columns]

# Mostra colunas relevantes
cand_cols = ["Valor_Venda", "Valor_Compra", "Data_Coleta"]
existing = [c for c in cand_cols if c in fuel_dataframe.columns]
print("Colunas de interesse encontradas:", existing)
show_head(fuel_dataframe[existing], 5, "Amostra (colunas renomeadas)")


# ------------------------------------------------------------
# 9) Ajuste de tipos: datas e numéricos
# ------------------------------------------------------------
print("\n9) Ajustando tipos de Data_Coleta / Valor_Venda / Valor_Compra")

if "Data_Coleta" in fuel_dataframe.columns:
    fuel_dataframe["Data_Coleta"] = ensure_datetime(fuel_dataframe["Data_Coleta"])

for col in ("Valor_Venda", "Valor_Compra"):
    if col in fuel_dataframe.columns:
        fuel_dataframe[col] = as_float_pt(fuel_dataframe[col])

show_head(fuel_dataframe[existing], 5, "Amostra pós-tipos")
print("Tipos:")
print(fuel_dataframe.dtypes)

# ------------------------------------------------------------
# 4–8) Inspeções diversas
# ------------------------------------------------------------
print("\n4) Primeiras 6 linhas:")
show_head(fuel_dataframe, 6)

print("\n5) Últimas 10 linhas:")
show_tail(fuel_dataframe, 10)

print("\n6) Dimensões (linhas, colunas):")
print(fuel_dataframe.shape)

print("\n7) Nomes das colunas:")
print(list(fuel_dataframe.columns))

print("\n8) Principais variáveis: ver comentários no início do script.")

# ------------------------------------------------------------
# 10) Seleciona duas colunas (ex.: Revenda, Valor_Venda)
# ------------------------------------------------------------
print("\n10) Selecionando duas colunas (Revenda, Valor_Venda)")
cols10 = [c for c in ["Revenda", "Valor_Venda"] if c in fuel_dataframe.columns]
if cols10:
    show_head(fuel_dataframe[cols10], 5)
else:
    print("Colunas não encontradas.")

# ------------------------------------------------------------
# 11) Filtro onde variável numérica >= 2
# ------------------------------------------------------------
print("\n11) Filtrando Valor_Venda >= 2")
if "Valor_Venda" in fuel_dataframe.columns:
    show_head(fuel_dataframe.query("Valor_Venda >= 2"), 5)
else:
    print("Valor_Venda não encontrada.")

# ------------------------------------------------------------
# 12) Ordena crescente por Valor_Venda
# ------------------------------------------------------------
print("\n12) Ordenando por Valor_Venda (crescente)")
if "Valor_Venda" in fuel_dataframe.columns:
    fuel_dataframe = fuel_dataframe.sort_values("Valor_Venda", ascending=True)
    show_head(fuel_dataframe, 5)
else:
    print("Valor_Venda não encontrada.")

# ------------------------------------------------------------
# 13) Nova coluna: Lucro_Venda = Valor_Venda - Valor_Compra
# ------------------------------------------------------------
print("\n13) Criando coluna Lucro_Venda")
if {"Valor_Venda", "Valor_Compra"}.issubset(fuel_dataframe.columns):
    fuel_dataframe["Lucro_Venda"] = fuel_dataframe["Valor_Venda"] - fuel_dataframe["Valor_Compra"]
    show_head(fuel_dataframe[["Lucro_Venda"]], 5)
else:
    print("Colunas Valor_Venda/Valor_Compra não encontradas.")

# ------------------------------------------------------------
# 14) Remove coluna 'Complemento' (se existir)
# ------------------------------------------------------------
print("\n14) Removendo coluna 'Complemento' (se existir)")
if "Complemento" in fuel_dataframe.columns:
    fuel_dataframe = fuel_dataframe.drop(columns=["Complemento"])
show_head(fuel_dataframe, 5)

# ------------------------------------------------------------
# 15) select 3 colunas (o código original mostra 2; mantemos 2 por segurança)
# ------------------------------------------------------------
print("\n15) Selecionando (Revenda, Valor_Venda)")
cols15 = [c for c in ["Revenda", "Valor_Venda"] if c in fuel_dataframe.columns]
if cols15:
    show_head(fuel_dataframe[cols15], 5)

# ------------------------------------------------------------
# 16) filter: Valor_Venda >= 2*Valor_Compra e Valor_Compra != 0
# ------------------------------------------------------------
print("\n16) Filtrando Valor_Venda >= 2*Valor_Compra & Valor_Compra != 0")
if {"Valor_Venda", "Valor_Compra"}.issubset(fuel_dataframe.columns):
    mask = (fuel_dataframe["Valor_Venda"] >= 2 * fuel_dataframe["Valor_Compra"]) & (fuel_dataframe["Valor_Compra"] != 0)
    show_head(fuel_dataframe.loc[mask], 5)

# ------------------------------------------------------------
# 17) Seleciona colunas que começam com 'V'
# ------------------------------------------------------------
print("\n17) Colunas que começam com 'V'")
vcols = [c for c in fuel_dataframe.columns if c.startswith("V")]
if vcols:
    show_head(fuel_dataframe[vcols], 5)
else:
    print("Nenhuma coluna iniciada por 'V'.")

# ------------------------------------------------------------
# 19) arrange desc(Data_Coleta)
# ------------------------------------------------------------
print("\n19) Ordenando por Data_Coleta (decrescente)")
if "Data_Coleta" in fuel_dataframe.columns:
    fuel_dataframe = fuel_dataframe.sort_values("Data_Coleta", ascending=False)
    show_head(fuel_dataframe, 5)
else:
    print("Data_Coleta não encontrada.")

# ------------------------------------------------------------
# 20) mutate Margem_Lucro = (Lucro_Venda/Valor_Venda)*100
# ------------------------------------------------------------
print("\n20) Criando coluna Margem_Lucro (%)")
if {"Lucro_Venda", "Valor_Venda"}.issubset(fuel_dataframe.columns):
    with np.errstate(divide="ignore", invalid="ignore"):
        fuel_dataframe["Margem_Lucro"] = np.where(
            fuel_dataframe["Valor_Venda"] != 0,
            (fuel_dataframe["Lucro_Venda"] / fuel_dataframe["Valor_Venda"]) * 100.0,
            np.nan,
        )
    show_head(fuel_dataframe[["Valor_Venda", "Valor_Compra", "Lucro_Venda", "Margem_Lucro"]], 5)

# ------------------------------------------------------------
# Subconjunto até 2005-07-31
# ------------------------------------------------------------
print("\nSubconjunto: Data_Coleta <= 2005-07-31")
df = fuel_dataframe.copy()
if "Data_Coleta" in df.columns:
    df = df.loc[df["Data_Coleta"] <= pd.Timestamp("2005-07-31")]

# ------------------------------------------------------------
# 21) resumo de coluna numérica
# ------------------------------------------------------------
print("\n21) Resumo de Valor_Venda (count, mean, std, min, median, max)")
if "Valor_Venda" in df.columns:
    s = df["Valor_Venda"]
    stats21 = {
        "count": int(s.notna().sum()),
        "mean": float(s.mean(skipna=True)) if len(s) else np.nan,
        "std": float(s.std(skipna=True)) if len(s) else np.nan,
        "min": float(s.min(skipna=True)) if len(s) else np.nan,
        "median": float(s.median(skipna=True)) if len(s) else np.nan,
        "max": float(s.max(skipna=True)) if len(s) else np.nan,
    }
    print(stats21)
else:
    print("Valor_Venda não encontrada no subconjunto.")

# ------------------------------------------------------------
# 22) groupby Produto -> média Valor_Venda
# ------------------------------------------------------------
print("\n22) Média de Valor_Venda por Produto (top 10)")
if {"Produto", "Valor_Venda"}.issubset(df.columns):
    grp22 = df.groupby("Produto", dropna=False)["Valor_Venda"].mean().reset_index(name="mean")
    show_head(grp22, 10)
else:
    print("Produto/Valor_Venda ausentes.")

# ------------------------------------------------------------
# 23) groupby Produto -> count/mean/median/std Valor_Venda
# ------------------------------------------------------------
print("\n23) Estatísticas de Valor_Venda por Produto (top 10 por média)")
if {"Produto", "Valor_Venda"}.issubset(df.columns):
    grp23 = (
        df.groupby("Produto", dropna=False)["Valor_Venda"]
        .agg(count=lambda x: int(x.notna().sum()),
             mean="mean",
             median="median",
             std="std")
        .reset_index()
        .sort_values("mean", ascending=False)
    )
    show_head(grp23, 10)

# ------------------------------------------------------------
# 24) pivot_longer -> melt
# ------------------------------------------------------------
print("\n24) pivot_longer de Valor_Venda / Valor_Compra")
nums_for_longer = [c for c in ["Valor_Venda", "Valor_Compra"] if c in df.columns]
if "Produto" in df.columns and len(nums_for_longer) == 2:
    long24 = df[["Produto"] + nums_for_longer].melt(id_vars=["Produto"], var_name="variavel", value_name="valor")
    show_head(long24, 10)

# ------------------------------------------------------------
# 25) select -> dropna -> arrange
# ------------------------------------------------------------
print("\n25) select Produto, Valor_Venda -> dropna -> sort by Valor_Venda")
if {"Produto", "Valor_Venda"}.issubset(df.columns):
    pipe25 = df[["Produto", "Valor_Venda"]].dropna().sort_values("Valor_Venda")
    show_head(pipe25, 10)

# ------------------------------------------------------------
# 26) pivot_wider: count(Produto, Estado_Sigla)
# ------------------------------------------------------------
print("\n26) pivot_wider de contagem por Produto x Estado_Sigla")
estado_col = first_existing(df, ["Estado_Sigla", "Estado_-_Sigla", "Estado__Sigla", "Estado_Sigla_"])
if estado_col is None and "Estado_-_Sigla" in df.columns:
    estado_col = "Estado_-_Sigla"
if estado_col is None and "Estado - Sigla" in df.columns:
    # caso extremo de não ter sido sanitizado
    estado_col = "Estado - Sigla"

if "Produto" in df.columns and estado_col is not None:
    wide26 = (
        df.groupby(["Produto", estado_col]).size().reset_index(name="n")
        .pivot(index="Produto", columns=estado_col, values="n")
        .fillna(0)
        .astype(int)
        .reset_index()
    )
    show_head(wide26, 5)
else:
    print("Colunas necessárias ausentes para o pivot_wider.")

# ------------------------------------------------------------
# 27) drop_na
# ------------------------------------------------------------
print("\n27) drop_na em df")
df_no_na = df.dropna()
show_head(df_no_na, 5)

# ------------------------------------------------------------
# 28) Substituir NA por 0 em Valor_Venda (nova coluna)
# ------------------------------------------------------------
print("\n28) Nova coluna valor_venda_filled0 (NA -> 0)")
if "Valor_Venda" in df.columns:
    df["valor_venda_filled0"] = df["Valor_Venda"].fillna(0.0)
    show_head(df[["Valor_Venda", "valor_venda_filled0"]], 5)

# ------------------------------------------------------------
# Gráficos (29–37) - cada um em sua figura; salvamos PNGs em ./plots
# ------------------------------------------------------------
def save_or_show(fig: plt.Figure, name: str) -> None:
    out = PLOTS_DIR / f"{name}.png"
    fig.savefig(out, bbox_inches="tight", dpi=140)
    print(f"Figura salva em: {out.as_posix()}")
    plt.close(fig)


# 29) Dispersão Valor_Venda vs Valor_Compra
print("\n29) Gráfico de dispersão: Valor_Venda vs Valor_Compra")
if {"Valor_Venda", "Valor_Compra"}.issubset(df.columns):
    fig = plt.figure()
    ax = fig.gca()
    ax.scatter(df["Valor_Venda"], df["Valor_Compra"], alpha=0.2)
    ax.set_title("Dispersão: Valor de Venda vs Valor de Compra")
    ax.set_xlabel("Valor de Venda")
    ax.set_ylabel("Valor de Compra")
    save_or_show(fig, "29_disp_venda_compra")

# 30) Barras: contagem por Produto (Top 20)
print("\n30) Barras: contagem por Produto (Top 20)")
if "Produto" in df.columns:
    cont = df["Produto"].value_counts().head(20)[::-1]  # invert para barra horizontal
    fig = plt.figure()
    ax = fig.gca()
    ax.barh(cont.index.astype(str), cont.values)
    ax.set_title("Contagem por Produto (Top 20)")
    ax.set_xlabel("Contagem")
    ax.set_ylabel("Produto")
    save_or_show(fig, "30_barras_produto_top20")

# 31) Histograma Valor_Venda
print("\n31) Histograma: Valor_Venda")
if "Valor_Venda" in df.columns:
    fig = plt.figure()
    ax = fig.gca()
    ax.hist(df["Valor_Venda"].dropna().values, bins=30)
    ax.set_title("Histograma: Valor de Venda")
    ax.set_xlabel("Valor de Venda")
    ax.set_ylabel("Frequência")
    save_or_show(fig, "31_hist_valor_venda")

# 32) Linha: evolução diária do Valor_Venda
print("\n32) Evolução diária do Valor_Venda")
if {"Data_Coleta", "Valor_Venda"}.issubset(df.columns):
    ts = (
        df.dropna(subset=["Data_Coleta"])
        .groupby(df["Data_Coleta"].dt.date)["Valor_Venda"].mean()
        .reset_index(name="venda_media")
    )
    fig = plt.figure()
    ax = fig.gca()
    ax.plot(pd.to_datetime(ts["Data_Coleta"]), ts["venda_media"])
    ax.set_title("Evolução diária do Valor de Venda")
    ax.set_xlabel("Data")
    ax.set_ylabel("Preço médio")
    save_or_show(fig, "32_linha_evolucao_venda")

# 33) Dispersão com tendência (reta de regressão simples)
print("\n33) Dispersão com linha de tendência")
if {"Valor_Venda", "Valor_Compra"}.issubset(df.columns):
    x = df["Valor_Venda"].values
    y = df["Valor_Compra"].values
    mask = np.isfinite(x) & np.isfinite(y)
    x_m = x[mask]
    y_m = y[mask]
    fig = plt.figure()
    ax = fig.gca()
    ax.scatter(x_m, y_m, alpha=0.2)
    if len(x_m) >= 2:
        # regressão linear simples y = a*x + b
        a, b = np.polyfit(x_m, y_m, 1)
        xs = np.linspace(np.nanmin(x_m), np.nanmax(x_m), 200)
        ax.plot(xs, a * xs + b)
    ax.set_title("Dispersão com Tendência: Venda vs Compra")
    ax.set_xlabel("Valor de Venda")
    ax.set_ylabel("Valor de Compra")
    save_or_show(fig, "33_disp_tendencia")

# 34) Boxplot por Produto (Top 8 por frequência)
print("\n34) Boxplot: Valor_Venda por Produto (Top 8)")
if {"Produto", "Valor_Venda"}.issubset(df.columns):
    top_prod = df["Produto"].value_counts().head(8).index.tolist()
    df_top = df[df["Produto"].isin(top_prod)].copy()
    # ordena produtos pela mediana desc
    med_ord = (
        df_top.groupby("Produto")["Valor_Venda"].median().sort_values(ascending=False).index.tolist()
    )
    df_top["Produto"] = pd.Categorical(df_top["Produto"], categories=med_ord, ordered=True)
    fig = plt.figure(figsize=(8, 5))
    ax = fig.gca()
    # boxplot por categoria
    data = [df_top.loc[df_top["Produto"] == p, "Valor_Venda"].dropna().values for p in med_ord]
    ax.boxplot(data, vert=False, labels=med_ord, whis=1.5, showfliers=True)
    ax.set_title("Boxplot: Valor de Venda por Produto (Top 8)")
    ax.set_xlabel("Valor de Venda (R$)")
    ax.set_ylabel("Produto")
    save_or_show(fig, "34_boxplot_produto")

# 35) Personalização de gráfico: barras por total de vendas (soma)
print("\n35) Valor total (soma) de Venda por Produto")
if {"Produto", "Valor_Venda"}.issubset(df.columns):
    total = df.groupby("Produto")["Valor_Venda"].sum().sort_values()[::-1]
    fig = plt.figure(figsize=(8, 6))
    ax = fig.gca()
    ax.barh(total.index.astype(str), total.values)
    ax.set_title("Valor Total de Vendas por Produto")
    ax.set_xlabel("Total de Vendas (R$)")
    ax.set_ylabel("Produto")
    save_or_show(fig, "35_total_vendas_produto")

# 36) Heatmap Produto x Bandeira (média de Valor_Venda)
print("\n36) Heatmap (rótulos numéricos) de média por Produto x Bandeira")
if {"Produto", "Bandeira", "Valor_Venda"}.issubset(df.columns):
    # limita 10 bandeiras principais
    top_n = 10
    top_b = df["Bandeira"].value_counts().head(top_n).index
    aux = (
        df.assign(Bandeira=np.where(df["Bandeira"].isin(top_b), df["Bandeira"], "Outras"))
        .groupby(["Produto", "Bandeira"])["Valor_Venda"]
        .mean()
        .unstack(fill_value=np.nan)
        .sort_index()
    )

    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    im = ax.imshow(aux.values, aspect="auto")
    ax.set_xticks(np.arange(aux.shape[1]))
    ax.set_xticklabels(list(aux.columns), rotation=45, ha="right")
    ax.set_yticks(np.arange(aux.shape[0]))
    ax.set_yticklabels(list(aux.index))
    ax.set_title("Heatmap: Média de Valor de Venda por Produto e Bandeira")
    ax.set_xlabel("Bandeira")
    ax.set_ylabel("Produto")

    # rótulos
    for i in range(aux.shape[0]):
        for j in range(aux.shape[1]):
            val = aux.values[i, j]
            if np.isfinite(val):
                ax.text(j, i, f"R$ {val:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."),
                        ha="center", va="center")

    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    save_or_show(fig, "36_heatmap_produto_bandeira")

# 37) facet_wrap análogo: 3 subplots separados (um por Produto top3)
print("\n37) Dispersão por Produto (Top 3)")
if {"Produto", "Valor_Venda", "Valor_Compra"}.issubset(df.columns):
    top3 = df["Produto"].value_counts().head(3).index.tolist()
    for prod in top3:
        sub = df[df["Produto"] == prod]
        fig = plt.figure()
        ax = fig.gca()
        ax.scatter(sub["Valor_Venda"], sub["Valor_Compra"], alpha=0.2)
        ax.set_title(f"Dispersão por Produto: {prod}")
        ax.set_xlabel("Valor de Venda")
        ax.set_ylabel("Valor de Compra")
        save_or_show(fig, f"37_disp_{sanitize_col(str(prod))}")

# ------------------------------------------------------------
# 38) Função resumo_variavel (com hist opcional)
# ------------------------------------------------------------
print("\n38) Função resumo_variavel()")

def resumo_variavel(dataframe: pd.DataFrame, colname: str, plot: bool = True) -> pd.DataFrame:
    assert colname in dataframe.columns, "Coluna não encontrada."
    s = dataframe[colname]
    if not np.issubdtype(s.dtype, np.number):
        raise TypeError("Coluna deve ser numérica.")
    tib = pd.DataFrame(
        {
            "min": [float(np.nanmin(s.values)) if len(s) else np.nan],
            "max": [float(np.nanmax(s.values)) if len(s) else np.nan],
            "mean": [float(np.nanmean(s.values)) if len(s) else np.nan],
            "median": [float(np.nanmedian(s.values)) if len(s) else np.nan],
            "sd": [float(np.nanstd(s.values, ddof=1)) if len(s) else np.nan],
            "n": [int(pd.notna(s).sum())],
        }
    )
    if plot:
        fig = plt.figure()
        ax = fig.gca()
        ax.hist(s.dropna().values, bins=30)
        ax.set_title(f"Histograma de {colname}")
        ax.set_xlabel(colname)
        ax.set_ylabel("Frequência")
        save_or_show(fig, f"38_hist_{sanitize_col(colname)}")
    return tib

if "Valor_Venda" in df.columns:
    print(resumo_variavel(df, "Valor_Venda", plot=False).to_string(index=False))

# ------------------------------------------------------------
# 39) Pipeline: ratio, stats por Produto
# ------------------------------------------------------------
print("\n39) Pipeline com ratio_venda_compra por Produto")
if {"Produto", "Valor_Venda", "Valor_Compra"}.issubset(df.columns):
    ratio = df[["Produto", "Valor_Venda", "Valor_Compra"]].dropna().copy()
    ratio["ratio_venda_compra"] = np.where(ratio["Valor_Compra"] != 0, ratio["Valor_Venda"] / ratio["Valor_Compra"], np.nan)
    pipeline39 = (
        ratio.groupby("Produto")["ratio_venda_compra"]
        .agg(media="mean", mediana="median", desvio="std")
        .reset_index()
        .sort_values("media", ascending=False)
    )
    show_head(pipeline39, 10)

# ------------------------------------------------------------
# 40) Pipeline: só numéricas, fill 0, GrupoNovo por média de referência
# ------------------------------------------------------------
print("\n40) Pipeline com todas as numéricas, preenchendo 0 e agrupando por 'Alto'/'Baixo'")
num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
if num_cols:
    ref = "Valor_Venda" if "Valor_Venda" in num_cols else num_cols[0]
    tmp = df[num_cols].copy()
    tmp = tmp.fillna(0)
    ref_mean = tmp[ref].mean()
    tmp["GrupoNovo"] = np.where(tmp[ref] > ref_mean, "Alto", "Baixo")
    pipeline40 = (
        tmp.groupby("GrupoNovo")
        .agg({c: ["mean", "median", "max"] for c in num_cols})
        .sort_values((ref, "mean"), ascending=False)
    )
    print(pipeline40.head(10))
else:
    print("Não há colunas numéricas no subconjunto.")

# ------------------------------------------------------------
# 41) Salvar pipeline (Q40) como função; carregar; executar
# ------------------------------------------------------------
print("\n41) Salvando funções em 'fuel_data_analysis.py', importando e executando no subconjunto")

MODULE_PATH = Path("fuel_data_analysis.py")
MODULE_PATH.write_text(
    """# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

def meu_pipeline(dataframe: pd.DataFrame) -> pd.DataFrame:
    num_cols = [c for c in dataframe.columns if np.issubdtype(dataframe[c].dtype, np.number)]
    if not num_cols:
        return pd.DataFrame()
    ref = "Valor_Venda" if "Valor_Venda" in num_cols else num_cols[0]
    tmp = dataframe[num_cols].copy().fillna(0)
    ref_mean = tmp[ref].mean()
    tmp["GrupoNovo"] = np.where(tmp[ref] > ref_mean, "Alto", "Baixo")
    out = (
        tmp.groupby("GrupoNovo")
        .agg({c: ['mean', 'median', 'max'] for c in num_cols})
        .sort_values((ref, 'mean'), ascending=False)
    )
    return out

def resumo_variavel(dataframe: pd.DataFrame, colname: str, plot: bool = True) -> pd.DataFrame:
    s = dataframe[colname]
    if not np.issubdtype(s.dtype, np.number):
        raise TypeError("Coluna deve ser numérica.")
    return pd.DataFrame({
        'min': [float(np.nanmin(s.values))],
        'max': [float(np.nanmax(s.values))],
        'mean': [float(np.nanmean(s.values))],
        'median': [float(np.nanmedian(s.values))],
        'sd': [float(np.nanstd(s.values, ddof=1))],
        'n': [int(pd.notna(s).sum())],
    })
"""
)

# Carrega dinamicamente
import importlib.util
spec = importlib.util.spec_from_file_location("fuel_data_analysis", MODULE_PATH.as_posix())
mod = importlib.util.module_from_spec(spec)
assert spec.loader is not None
spec.loader.exec_module(mod)

resultado_final = mod.meu_pipeline(df if not df.empty else fuel_dataframe)
print("\nResultado final do meu_pipeline (topo):")
try:
    print(resultado_final.head(10))
except Exception:
    print(resultado_final)

print("\nConcluído com sucesso. Gráficos (quando aplicáveis) foram salvos na pasta ./plots.")


# 0) Renomeando arquivos fora do padrão para ca-AAAA-MM.csv
Nenhum arquivo fora do padrão encontrado.

1) Importando CSVs do diretório ./dataset (recursivo)
 - dataset/ca-2005-01.csv
 - dataset/ca-2005-02.csv
 - dataset/ca-2006-01.csv
 - dataset/ca-2006-02.csv
 - dataset/ca-2007-01.csv


Erro ao ler dataset/ca-2014-01.csv: NULL byte detected. This byte cannot be processed in Python's native csv library at the moment, so please pass in engine='c' instead


Total de linhas após concatenação: 22,766,825

Pré-rename (amostra)
ï»¿Regiao - Sigla Estado - Sigla Municipio                                                        Revenda     CNPJ da Revenda              Nome da Rua Numero Rua         Complemento         Bairro       Cep  Produto Data da Coleta Valor de Venda Valor de Compra Unidade de Medida                     Bandeira Regiao - Sigla
               SE             SP GUARULHOS                                       AUTO POSTO SAKAMOTO LTDA  49.051.667/0001-02 RODOVIA PRESIDENTE DUTRA        S/N KM 210,5-SENT SP/RJ     BONSUCESSO 07178-580 GASOLINA     04/01/2005          2,257          1,9594        R$ / litro PETROBRAS DISTRIBUIDORA S.A.            NaN
               SE             SP GUARULHOS                                       AUTO POSTO SAKAMOTO LTDA  49.051.667/0001-02 RODOVIA PRESIDENTE DUTRA        S/N KM 210,5-SENT SP/RJ     BONSUCESSO 07178-580   ETANOL     04/01/2005          1,449          1,0962        R$ / litro PETR

/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2118040827.py:536: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, vert=False, labels=med_ord, whis=1.5, showfliers=True)


Figura salva em: plots/34_boxplot_produto.png

35) Valor total (soma) de Venda por Produto
Figura salva em: plots/35_total_vendas_produto.png

36) Heatmap (rótulos numéricos) de média por Produto x Bandeira
Figura salva em: plots/36_heatmap_produto_bandeira.png

37) Dispersão por Produto (Top 3)
Figura salva em: plots/37_disp_GASOLINA.png
Figura salva em: plots/37_disp_ETANOL.png
Figura salva em: plots/37_disp_DIESEL.png

38) Função resumo_variavel()
  min  max     mean  median       sd       n
0.659 3.22 1.816352     1.7 0.420895 1052978

39) Pipeline com ratio_venda_compra por Produto
 Produto    media  mediana   desvio
     GNV 1.612981 1.427755 0.371309
  ETANOL 1.202906 1.181292 0.130510
GASOLINA 1.153119 1.152756 0.052285
  DIESEL 1.123361 1.120678 0.040904

40) Pipeline com todas as numéricas, preenchendo 0 e agrupando por 'Alto'/'Baixo'
          Valor_Venda               Valor_Compra                 Lucro_Venda                 Margem_Lucro                   valor_venda_filled0

In [15]:
## Análise do Período de Pandemia

#A partir deste ponto, serão realizadas análises específicas relacionadas ao período de pandemia, 
#utilizando os dados filtrados e categorizados para este intervalo de tempo. As análises buscarão identificar padrões,
#tendências e insights relevantes para este período.

In [16]:
import pandas as pd
import numpy as np

df = fuel_dataframe.copy()

# garante tipos
df["Data_Coleta"] = pd.to_datetime(df["Data_Coleta"], errors="coerce")
df = df.dropna(subset=["Data_Coleta","Produto","Valor_Venda"])

bins = [pd.Timestamp("1900-01-01"),
        pd.Timestamp("2020-02-29"),
        pd.Timestamp("2021-12-31"),
        pd.Timestamp("2100-01-01")]
labels = ["pre", "pandemia", "pos"]

df["periodo"] = pd.cut(df["Data_Coleta"], bins=bins, labels=labels, right=True)


Nível de preço e tendência
O que ver: como o preço médio mudou por período e por combustível; se a inclinação durante a pandemia foi anormal.

In [17]:
# média mensal por produto
m = (df
     .assign(mes = df["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
     .groupby(["Produto","mes","periodo"], dropna=False)["Valor_Venda"]
     .mean()
     .reset_index())

# tabela resumo por período
nivel = (m.groupby(["Produto","periodo"])["Valor_Venda"]
           .agg(media="mean", mediana="median")
           .reset_index())
print(nivel)


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2496800786.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  m = (df


               Produto   periodo     media   mediana
0               DIESEL       pre  2.455073  2.125600
1               DIESEL  pandemia  4.068810  3.947368
2               DIESEL       pos  6.133782  6.061685
3           DIESEL S10       pre  3.149423  3.150066
4           DIESEL S10  pandemia  4.138262  4.010615
5           DIESEL S10       pos  6.215221  6.119060
6           DIESEL S50       pre  2.151107  2.122141
7           DIESEL S50  pandemia       NaN       NaN
8           DIESEL S50       pos       NaN       NaN
9               ETANOL       pre  2.202719  2.105242
10              ETANOL  pandemia  3.952228  3.634678
11              ETANOL       pos  4.293974  4.171259
12            GASOLINA       pre  3.112262  2.773623
13            GASOLINA  pandemia  5.113872  4.912242
14            GASOLINA       pos  5.919475  5.851726
15  GASOLINA ADITIVADA       pre       NaN       NaN
16  GASOLINA ADITIVADA  pandemia  5.648859  5.740193
17  GASOLINA ADITIVADA       pos  6.100695  6.

/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2496800786.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  nivel = (m.groupby(["Produto","periodo"])["Valor_Venda"]


Volatilidade (incerteza)
O que ver: a dispersão (desvio-padrão / coeficiente de variação) por período.

In [18]:
vol = (m.groupby(["Produto","periodo"])["Valor_Venda"]
         .agg(std="std", media="mean")
         .assign(cv=lambda t: t["std"]/t["media"])
         .reset_index())
print(vol)

               Produto   periodo       std     media        cv
0               DIESEL       pre  0.627219  2.455073  0.255479
1               DIESEL  pandemia  0.730106  4.068810  0.179440
2               DIESEL       pos  0.511381  6.133782  0.083371
3           DIESEL S10       pre  0.449254  3.149423  0.142647
4           DIESEL S10  pandemia  0.721051  4.138262  0.174240
5           DIESEL S10       pos  0.523042  6.215221  0.084155
6           DIESEL S50       pre  0.066518  2.151107  0.030923
7           DIESEL S50  pandemia       NaN       NaN       NaN
8           DIESEL S50       pos       NaN       NaN       NaN
9               ETANOL       pre  0.597233  2.202719  0.271135
10              ETANOL  pandemia  0.866489  3.952228  0.219241
11              ETANOL       pos  0.488973  4.293974  0.113874
12            GASOLINA       pre  0.705034  3.112262  0.226534
13            GASOLINA  pandemia  0.922861  5.113872  0.180462
14            GASOLINA       pos  0.610210  5.919475  0

/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2476774477.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vol = (m.groupby(["Produto","periodo"])["Valor_Venda"]


Margem e repasse (se você tem Valor_Compra)
O que ver: mudou a margem? O repasse foi mais rápido para cima do que para baixo?

In [19]:
ok = {"Valor_Venda", "Valor_Compra"}.issubset(df.columns)
if ok:
    df["Lucro_Venda"] = df["Valor_Venda"] - df["Valor_Compra"]
    df["Margem_%"] = np.where(
        df["Valor_Venda"] > 0,
        (df["Lucro_Venda"] / df["Valor_Venda"]) * 100.0,
        np.nan
    )

    # ✅ forma 1: selecionando várias colunas corretamente (duas chaves)
    margem = (
        df.groupby(["Produto", "periodo"])[["Lucro_Venda", "Margem_%"]]
          .mean()
          .reset_index()
    )
    print(margem)

    # ✅ forma 2: equivalente usando .agg (evita qualquer ambiguidade)
    margem2 = (
        df.groupby(["Produto", "periodo"])
          .agg(Lucro_Venda_media=("Lucro_Venda", "mean"),
               Margem_media_pct=("Margem_%", "mean"))
          .reset_index()
    )
    print(margem2)
else:
    print("Colunas Valor_Venda e/ou Valor_Compra não encontradas.")

/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/3405097585.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["Produto", "periodo"])[["Lucro_Venda", "Margem_%"]]


               Produto   periodo  Lucro_Venda    Margem_%
0               DIESEL       pre     1.250502   53.947376
1               DIESEL  pandemia     3.753074   91.787930
2               DIESEL       pos     6.132010  100.000000
3           DIESEL S10       pre     2.096734   64.706242
4           DIESEL S10  pandemia     3.899631   93.039924
5           DIESEL S10       pos     6.234145  100.000000
6           DIESEL S50       pre     1.220378   56.136756
7           DIESEL S50  pandemia          NaN         NaN
8           DIESEL S50       pos          NaN         NaN
9               ETANOL       pre     1.242785   58.328608
10              ETANOL  pandemia     3.752177   92.775413
11              ETANOL       pos     4.334042  100.000000
12            GASOLINA       pre     1.597275   52.491512
13            GASOLINA  pandemia     4.809011   92.434195
14            GASOLINA       pos     5.974131  100.000000
15  GASOLINA ADITIVADA       pre          NaN         NaN
16  GASOLINA A

/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/3405097585.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["Produto", "periodo"])


               Produto   periodo  Lucro_Venda_media  Margem_media_pct
0               DIESEL       pre           1.250502         53.947376
1               DIESEL  pandemia           3.753074         91.787930
2               DIESEL       pos           6.132010        100.000000
3           DIESEL S10       pre           2.096734         64.706242
4           DIESEL S10  pandemia           3.899631         93.039924
5           DIESEL S10       pos           6.234145        100.000000
6           DIESEL S50       pre           1.220378         56.136756
7           DIESEL S50  pandemia                NaN               NaN
8           DIESEL S50       pos                NaN               NaN
9               ETANOL       pre           1.242785         58.328608
10              ETANOL  pandemia           3.752177         92.775413
11              ETANOL       pos           4.334042        100.000000
12            GASOLINA       pre           1.597275         52.491512
13            GASOLI

5) Dispersão geográfica
O que ver: estados/municípios onde o preço ficou mais alto ou mais volátil em cada período.

In [20]:
col_uf = [c for c in df.columns if c.lower().startswith("estado") and "sigla" in c.lower()]
uf = col_uf[0] if col_uf else None

if uf:
    geo = (df.groupby(["Produto","periodo",uf])["Valor_Venda"]
             .agg(media="mean", std="std", cv=lambda s: s.std()/s.mean())
             .reset_index())
    # top 5 estados mais caros em cada período/produto
    top5 = (geo.sort_values(["Produto","periodo","media"], ascending=[True,True,False])
               .groupby(["Produto","periodo"]).head(5))
    print(top5)


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2179141956.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  geo = (df.groupby(["Produto","periodo",uf])["Valor_Venda"]


    Produto periodo Estado_Sigla     media       std        cv
0    DIESEL     pre           AC  2.900635  0.732969  0.252693
12   DIESEL     pre           MT  2.569971  0.623740  0.242703
20   DIESEL     pre           RO  2.542283  0.600397  0.236164
21   DIESEL     pre           RR  2.506469  0.414040  0.165188
3    DIESEL     pre           AP  2.500664  0.675674  0.270198
..      ...     ...          ...       ...       ...       ...
546     GNV     pos           DF  6.635000  0.311395  0.046932
553     GNV     pos           PA  5.500000       NaN       NaN
557     GNV     pos           PR  5.258510  0.491166  0.093404
563     GNV     pos           SC  5.255417  0.368040  0.070031
549     GNV     pos           MA  5.230000  0.230000  0.043977

[105 rows x 6 columns]


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2179141956.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  top5 = (geo.sort_values(["Produto","periodo","media"], ascending=[True,True,False])


6) Relações entre combustíveis (spreads e razões)
O que ver:
Etanol/Gasolina (regra prática ~0,70 para flex)
Diesel–Gasolina spread
GNV/Gasolina

In [21]:
import unicodedata

In [22]:
# 6) Relações entre combustíveis (spreads e razões) — versão robusta

def _norm(s: str) -> str:
    # lower + remove acentos + compacta espaços
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    return " ".join(s.lower().split())

# médias mensais por produto (mantém 'periodo')
mix = (df.assign(mes=df["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
         .groupby(["mes","periodo","Produto"])["Valor_Venda"]
         .mean()
         .unstack("Produto"))

if mix.empty:
    print("Sem dados agregados para calcular relações.")
else:
    # mapeia colunas normalizadas -> originais
    cols = list(mix.columns)
    norm_map = {c: _norm(c) for c in cols}

    # famílias de produtos (palavras-chave)
    keys = {
        "gasolina": ["gasolina"],
        "etanol":   ["etanol", "alcool"],
        "diesel":   ["diesel", "oleo diesel"],
        "gnv":      ["gnv", "gas natural veicular", "gas natural"],
    }

    def family_columns(fam: str):
        kws = keys[fam]
        return [c for c in cols if any(k in norm_map[c] for k in kws)]

    gas_cols   = family_columns("gasolina")
    eta_cols   = family_columns("etanol")
    diesel_cols= family_columns("diesel")
    gnv_cols   = family_columns("gnv")

    print("Colunas detectadas por família:")
    print("  Gasolina:", gas_cols or "—")
    print("  Etanol  :", eta_cols or "—")
    print("  Diesel  :", diesel_cols or "—")
    print("  GNV     :", gnv_cols or "—")

    # cria séries agregadas por família (média das variantes quando houver mais de uma)
    fam = {}
    if gas_cols:   fam["gasolina"] = mix[gas_cols].mean(axis=1, skipna=True)
    if eta_cols:   fam["etanol"]   = mix[eta_cols].mean(axis=1, skipna=True)
    if diesel_cols:fam["diesel"]   = mix[diesel_cols].mean(axis=1, skipna=True)
    if gnv_cols:   fam["gnv"]      = mix[gnv_cols].mean(axis=1, skipna=True)

    out = pd.concat(fam, axis=1) if fam else pd.DataFrame(index=mix.index)

    # calcula relações apenas quando as duas pontas existem
    if {"etanol","gasolina"}.issubset(out.columns):
        out["ratio_etanol_gas"] = out["etanol"] / out["gasolina"]
    if {"diesel","gasolina"}.issubset(out.columns):
        out["spread_diesel_gas"] = out["diesel"] - out["gasolina"]
    if {"gnv","gasolina"}.issubset(out.columns):
        out["ratio_gnv_gas"] = out["gnv"] / out["gasolina"]

    # resumo por período
    if not out.empty:
        resumo = (out.reset_index()
                     .groupby("periodo")[["ratio_etanol_gas","spread_diesel_gas","ratio_gnv_gas"]]
                     .mean())
        print("\nMédias por período (quando disponíveis):")
        print(resumo)
    else:
        print("Não foi possível formar séries por família com os nomes encontrados.")


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2226819388.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mix = (df.assign(mes=df["Data_Coleta"].dt.to_period("M").dt.to_timestamp())


Colunas detectadas por família:
  Gasolina: ['GASOLINA', 'GASOLINA ADITIVADA']
  Etanol  : ['ETANOL']
  Diesel  : ['DIESEL', 'DIESEL S10', 'DIESEL S50']
  GNV     : ['GNV']

Médias por período (quando disponíveis):
          ratio_etanol_gas  spread_diesel_gas  ratio_gnv_gas
periodo                                                     
pre               0.701733          -0.628239       0.608219
pandemia          0.760396          -1.063402       0.690101
pos               0.715059           0.164416       0.808536


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/2226819388.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  resumo = (out.reset_index()


7) Tempo de “recuperação” para nível pré
O que ver: quanto tempo levou para cada combustível voltar ao patamar pré-pandemia (ou estabilizar abaixo/ao redor).

In [23]:
base = (m[m["periodo"]=="pre"]
        .groupby("Produto")["Valor_Venda"].median().rename("pre_mediana"))

rec = []
for prod, med in base.items():
    pos = m[(m["Produto"]==prod) & (m["periodo"]=="pos")].copy()
    if pos.empty: 
        continue
    # 3 meses seguidos <= 5% acima do nível pré (exemplo)
    pos["ok"] = pos["Valor_Venda"] <= 1.05*med
    pos["run"] = pos["ok"].rolling(3, min_periods=3).sum()
    hit = pos.loc[pos["run"]>=3, "mes"].min()
    rec.append({"Produto": prod, "pre_mediana": med, "data_recuperacao": hit})

print(pd.DataFrame(rec))


              Produto  pre_mediana data_recuperacao
0              DIESEL     2.125600              NaT
1          DIESEL S10     3.150066              NaT
2          DIESEL S50     2.122141              NaT
3              ETANOL     2.105242              NaT
4            GASOLINA     2.773623              NaT
5  GASOLINA ADITIVADA          NaN              NaT
6                 GNV     1.751386              NaT


In [24]:
# ===================== GRÁFICOS POR PERÍODO =====================
# Pré: <= 2020-02-29 | Pandemia: 2020-03-01..2021-12-31 | Pós: >= 2022-01-01
# Requer: df com colunas ["Data_Coleta","Produto","Valor_Venda"] e a coluna 'periodo' já criada
# Salva PNGs em ./plots

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import unicodedata

PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# Garante tipos
df = fuel_dataframe.copy()
df["Data_Coleta"] = pd.to_datetime(df.get("Data_Coleta"), errors="coerce")
df["Valor_Venda"] = pd.to_numeric(df.get("Valor_Venda"), errors="coerce")
if "Valor_Compra" in df.columns:
    df["Valor_Compra"] = pd.to_numeric(df["Valor_Compra"], errors="coerce")

# Cria 'periodo' se ainda não existir
if "periodo" not in df.columns:
    bins = [pd.Timestamp("1900-01-01"),
            pd.Timestamp("2020-02-29"),
            pd.Timestamp("2021-12-31"),
            pd.Timestamp("2100-01-01")]
    labels = ["pre", "pandemia", "pos"]
    df["periodo"] = pd.cut(df["Data_Coleta"], bins=bins, labels=labels, right=True)

# Agregado mensal por produto e período
m = (df.dropna(subset=["Data_Coleta","Produto","Valor_Venda"])
       .assign(mes=lambda t: t["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
       .groupby(["Produto","mes","periodo"], dropna=False)["Valor_Venda"]
       .mean()
       .reset_index())

periodos = ["pre","pandemia","pos"]
produtos = list(m["Produto"].dropna().unique())

def savefig(fig, name):
    out = PLOTS_DIR / name
    fig.savefig(out, bbox_inches="tight", dpi=140)
    print(f"Figura salva: {out}")
    plt.close(fig)

# 1) Séries temporais mensais por produto (uma figura por produto, linhas por período)
for prod in produtos:
    sub = m[m["Produto"] == prod]
    if sub.empty: 
        continue
    fig = plt.figure()
    ax = fig.gca()
    for per in periodos:
        s = sub[sub["periodo"] == per]
        if not s.empty:
            ax.plot(s["mes"], s["Valor_Venda"], label=per)
    ax.set_title(f"Evolução mensal do preço – {prod}")
    ax.set_xlabel("Mês")
    ax.set_ylabel("Preço médio (R$)")
    ax.legend()
    savefig(fig, f"serie_{unicodedata.normalize('NFKD', str(prod)).encode('ascii','ignore').decode('ascii')}.png")

# 2) Boxplots preço por período (uma figura por produto)
for prod in produtos:
    base = df[(df["Produto"] == prod) & df["Valor_Venda"].notna()]
    if base.empty: 
        continue
    data = [base.loc[base["periodo"]==p, "Valor_Venda"].dropna().values for p in periodos]
    # Se todas vazias, pula
    if all(len(x)==0 for x in data):
        continue
    fig = plt.figure()
    ax = fig.gca()
    ax.boxplot(data, labels=periodos, showfliers=True)
    ax.set_title(f"Boxplot por período – {prod}")
    ax.set_xlabel("Período")
    ax.set_ylabel("Preço (R$)")
    savefig(fig, f"boxplot_{unicodedata.normalize('NFKD', str(prod)).encode('ascii','ignore').decode('ascii')}.png")

# 3) Volatilidade (desvio-padrão) por período – uma figura por produto
vol = (m.groupby(["Produto","periodo"])["Valor_Venda"]
         .agg(std="std")
         .reset_index())
for prod in produtos:
    sub = vol[vol["Produto"] == prod]
    if sub.empty: 
        continue
    fig = plt.figure()
    ax = fig.gca()
    vals = [float(sub.loc[sub["periodo"]==p, "std"].values[0]) if (sub["periodo"]==p).any() else np.nan for p in periodos]
    ax.bar(periodos, vals)
    ax.set_title(f"Volatilidade (desvio-padrão) – {prod}")
    ax.set_xlabel("Período")
    ax.set_ylabel("Desvio-padrão do preço")
    savefig(fig, f"vol_{unicodedata.normalize('NFKD', str(prod)).encode('ascii','ignore').decode('ascii')}.png")

# 4) Margem e repasse (se houver Valor_Compra) – barra da margem média por período
if {"Valor_Venda","Valor_Compra"}.issubset(df.columns):
    df["Lucro_Venda"] = df["Valor_Venda"] - df["Valor_Compra"]
    df["Margem_%"] = np.where(df["Valor_Venda"]>0, df["Lucro_Venda"]/df["Valor_Venda"]*100, np.nan)
    marg = (df.groupby(["Produto","periodo"])["Margem_%"]
              .mean()
              .reset_index())
    for prod in produtos:
        sub = marg[marg["Produto"] == prod]
        if sub.empty: 
            continue
        fig = plt.figure()
        ax = fig.gca()
        vals = [float(sub.loc[sub["periodo"]==p, "Margem_%"].values[0]) if (sub["periodo"]==p).any() else np.nan for p in periodos]
        ax.bar(periodos, vals)
        ax.set_title(f"Margem média (%) por período – {prod}")
        ax.set_xlabel("Período")
        ax.set_ylabel("Margem (%)")
        savefig(fig, f"margem_{unicodedata.normalize('NFKD', str(prod)).encode('ascii','ignore').decode('ascii')}.png")

# 5) Razões/spreads entre combustíveis (robusto a nomes diferentes)
def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    return " ".join(s.lower().split())

mix = (df.assign(mes=df["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
         .groupby(["mes","periodo","Produto"])["Valor_Venda"]
         .mean()
         .unstack("Produto"))

if not mix.empty:
    cols = list(mix.columns)
    norm_map = {c: _norm(c) for c in cols}
    keys = {
        "gasolina": ["gasolina"],
        "etanol":   ["etanol","alcool"],
        "diesel":   ["diesel","oleo diesel"],
        "gnv":      ["gnv","gas natural veicular","gas natural"]
    }
    def family_columns(fam):
        return [c for c in cols if any(k in norm_map[c] for k in keys[fam])]

    fam_series = {}
    for fam in ["gasolina","etanol","diesel","gnv"]:
        fam_cols = family_columns(fam)
        if fam_cols:
            fam_series[fam] = mix[fam_cols].mean(axis=1, skipna=True)

    out = pd.concat(fam_series, axis=1) if fam_series else pd.DataFrame(index=mix.index)

    if not out.empty:
        out = out.reset_index()  # traz 'mes' e 'periodo' para colunas
        # Razão etanol/gasolina (com linha de referência 0,70)
        if {"etanol","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["etanol"]/s["gasolina"], label=per)
            ax.axhline(0.70, linestyle="--")
            ax.set_title("Razão Etanol/Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Razão")
            ax.legend()
            savefig(fig, "ratio_etanol_gasolina.png")

        # Spread Diesel - Gasolina
        if {"diesel","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["diesel"] - s["gasolina"], label=per)
            ax.set_title("Spread Diesel - Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Diferença (R$)")
            ax.legend()
            savefig(fig, "spread_diesel_gasolina.png")

        # Razão GNV/Gasolina
        if {"gnv","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["gnv"]/s["gasolina"], label=per)
            ax.set_title("Razão GNV/Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Razão")
            ax.legend()
            savefig(fig, "ratio_gnv_gasolina.png")

# 6) (Opcional) Heatmap Produto x UF por período específico
#    Chame a função abaixo passando um período: "pre", "pandemia" ou "pos"
def heatmap_produto_uf(periodo_escolhido: str):
    # tenta detectar a coluna UF
    uf_col = next((c for c in df.columns if "sigla" in c.lower() and "estado" in c.lower()), None)
    if uf_col is None:
        print("Não encontrei coluna de UF ('Estado - Sigla').")
        return
    base = df[(df["periodo"] == periodo_escolhido) & df["Valor_Venda"].notna()]
    if base.empty:
        print(f"Sem dados para período {periodo_escolhido}.")
        return
    tab = (base.groupby(["Produto", uf_col])["Valor_Venda"]
                .mean()
                .unstack(uf_col)
                .sort_index())
    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    im = ax.imshow(tab.values, aspect="auto")
    ax.set_xticks(np.arange(tab.shape[1])); ax.set_xticklabels(list(tab.columns), rotation=45, ha="right")
    ax.set_yticks(np.arange(tab.shape[0])); ax.set_yticklabels(list(tab.index))
    ax.set_title(f"Heatmap – preço médio por Produto x UF ({periodo_escolhido})")
    ax.set_xlabel("UF"); ax.set_ylabel("Produto")
    # rótulos
    for i in range(tab.shape[0]):
        for j in range(tab.shape[1]):
            val = tab.values[i, j]
            if np.isfinite(val):
                ax.text(j, i, f"R$ {val:,.2f}".replace(",", "X").replace(".", ",").replace("X","."), 
                        ha="center", va="center")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    savefig(fig, f"heatmap_produto_uf_{periodo_escolhido}.png")

# Exemplo de uso (descomente se quiser gerar agora):
# heatmap_produto_uf("pre")
# heatmap_produto_uf("pandemia")
# heatmap_produto_uf("pos")
# =================== FIM DOS GRÁFICOS POR PERÍODO ===================


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/556517805.py:32: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  m = (df.dropna(subset=["Data_Coleta","Produto","Valor_Venda"])


Figura salva: plots/serie_DIESEL.png
Figura salva: plots/serie_DIESEL S10.png
Figura salva: plots/serie_DIESEL S50.png
Figura salva: plots/serie_ETANOL.png
Figura salva: plots/serie_GASOLINA.png
Figura salva: plots/serie_GASOLINA ADITIVADA.png
Figura salva: plots/serie_GNV.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/556517805.py:75: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=periodos, showfliers=True)


Figura salva: plots/boxplot_DIESEL.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/556517805.py:75: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=periodos, showfliers=True)


...

Figura salva: plots/boxplot_GNV.png
Figura salva: plots/vol_DIESEL.png
Figura salva: plots/vol_DIESEL S10.png
Figura salva: plots/vol_DIESEL S50.png
Figura salva: plots/vol_ETANOL.png
Figura salva: plots/vol_GASOLINA.png
Figura salva: plots/vol_GASOLINA ADITIVADA.png
Figura salva: plots/vol_GNV.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/556517805.py:102: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  marg = (df.groupby(["Produto","periodo"])["Margem_%"]


Figura salva: plots/margem_DIESEL.png
Figura salva: plots/margem_DIESEL S10.png
Figura salva: plots/margem_DIESEL S50.png
Figura salva: plots/margem_ETANOL.png
Figura salva: plots/margem_GASOLINA.png
Figura salva: plots/margem_GASOLINA ADITIVADA.png
Figura salva: plots/margem_GNV.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/556517805.py:123: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mix = (df.assign(mes=df["Data_Coleta"].dt.to_period("M").dt.to_timestamp())


Figura salva: plots/ratio_etanol_gasolina.png
Figura salva: plots/spread_diesel_gasolina.png
Figura salva: plots/ratio_gnv_gasolina.png


In [ ]:
# =================== GRÁFICOS POR PERÍODO (funções reutilizáveis) ===================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import unicodedata

PLOTS_DIR = Path("plots")
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

def _slug(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    s = "".join(ch if ch.isalnum() or ch in ("-","_") else "_" for ch in s)
    while "__" in s: s = s.replace("__","_")
    return s.strip("_").lower()

def _savefig(fig, name, title_prefix=None):
    if title_prefix:
        ax = fig.gca()
        ax.set_title(f"{title_prefix} — {ax.get_title()}")
    out = PLOTS_DIR / name
    fig.savefig(out, bbox_inches="tight", dpi=140)
    print(f"Figura salva: {out}")
    plt.close(fig)

# --- Obtenção do dataframe base ---
if 'fuel_dataframe' in globals():
    _base = fuel_dataframe.copy()
elif 'df' in globals():
    _base = df.copy()
else:
    raise RuntimeError("Não encontrei 'fuel_dataframe' nem 'df' no ambiente. Execute a etapa de carga/limpeza antes.")

# Garantias de tipo
_base["Data_Coleta"] = pd.to_datetime(_base.get("Data_Coleta"), errors="coerce")
_base["Valor_Venda"] = pd.to_numeric(_base.get("Valor_Venda"), errors="coerce")
if "Valor_Compra" in _base.columns:
    _base["Valor_Compra"] = pd.to_numeric(_base.get("Valor_Compra"), errors="coerce")


if "periodo" not in _base.columns:
    bins = [pd.Timestamp("1900-01-01"),
            pd.Timestamp("2020-02-29"),
            pd.Timestamp("2021-12-31"),
            pd.Timestamp("2100-01-01")]
    labels = ["pre", "pandemia", "pos"]
    _base["periodo"] = pd.cut(_base["Data_Coleta"], bins=bins, labels=labels, right=True)

# Agregado mensal por produto e período
m = (_base.dropna(subset=["Data_Coleta","Produto","Valor_Venda"])
         .assign(mes=lambda t: t["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
         .groupby(["Produto","mes","periodo"], dropna=False)["Valor_Venda"]
         .mean()
         .reset_index())

periodos = ["pre","pandemia","pos"]
produtos = list(m["Produto"].dropna().unique())

# ============ (Item 2) Séries temporais mensais por produto ============
for prod in produtos:
    sub = m[m["Produto"] == prod]
    if sub.empty:
        continue
    fig = plt.figure()
    ax = fig.gca()
    for per in periodos:
        s = sub[sub["periodo"] == per]
        if not s.empty:
            ax.plot(s["mes"], s["Valor_Venda"], label=per)
    ax.set_title(f"Evolução mensal do preço – {prod}")
    ax.set_xlabel("Mês")
    ax.set_ylabel("Preço médio (R$)")
    ax.legend()
    _savefig(fig, f"item02_serie_{_slug(prod)}.png", title_prefix="Item 2")

# ============ (Item 2) Boxplots de preço por período ==============
for prod in produtos:
    base = _base[(_base["Produto"] == prod) & _base["Valor_Venda"].notna()]
    if base.empty:
        continue
    data = [base.loc[base["periodo"]==p, "Valor_Venda"].dropna().values for p in periodos]
    if all(len(x)==0 for x in data):
        continue
    fig = plt.figure()
    ax = fig.gca()
    ax.boxplot(data, labels=periodos, showfliers=True)
    ax.set_title(f"Boxplot por período – {prod}")
    ax.set_xlabel("Período")
    ax.set_ylabel("Preço (R$)")
    _savefig(fig, f"item02_boxplot_{_slug(prod)}.png", title_prefix="Item 2")

# ============ (Item 3) Volatilidade (desvio-padrão) por período ============
vol = (m.groupby(["Produto","periodo"])["Valor_Venda"]
         .agg(std="std")
         .reset_index())
for prod in produtos:
    sub = vol[vol["Produto"] == prod]
    if sub.empty: 
        continue
    fig = plt.figure()
    ax = fig.gca()
    vals = [float(sub.loc[sub["periodo"]==p, "std"].values[0]) if (sub["periodo"]==p).any() else np.nan for p in periodos]
    ax.bar(periodos, vals)
    ax.set_title(f"Volatilidade (desvio-padrão) – {prod}")
    ax.set_xlabel("Período")
    ax.set_ylabel("Desvio-padrão do preço")
    _savefig(fig, f"item03_vol_{_slug(prod)}.png", title_prefix="Item 3")

# ============ (Item 4) Margem média por período (se houver Valor_Compra) ============
if {"Valor_Venda","Valor_Compra"}.issubset(_base.columns):
    _base["Lucro_Venda"] = _base["Valor_Venda"] - _base["Valor_Compra"]
    _base["Margem_%"] = np.where(_base["Valor_Venda"]>0, _base["Lucro_Venda"]/_base["Valor_Venda"]*100, np.nan)
    marg = (_base.groupby(["Produto","periodo"])["Margem_%"]
              .mean()
              .reset_index())
    for prod in produtos:
        sub = marg[marg["Produto"] == prod]
        if sub.empty:
            continue
        fig = plt.figure()
        ax = fig.gca()
        vals = [float(sub.loc[sub["periodo"]==p, "Margem_%"].values[0]) if (sub["periodo"]==p).any() else np.nan for p in periodos]
        ax.bar(periodos, vals)
        ax.set_title(f"Margem média (%) por período – {prod}")
        ax.set_xlabel("Período")
        ax.set_ylabel("Margem (%)")
        _savefig(fig, f"item04_margem_{_slug(prod)}.png", title_prefix="Item 4")

# ============ (Item 5) Heatmap Produto x UF por período (função) ============
def heatmap_produto_uf(periodo_escolhido: str):
    uf_col = next((c for c in _base.columns if "sigla" in c.lower() and "estado" in c.lower()), None)
    if uf_col is None:
        print("Não encontrei coluna de UF ('Estado - Sigla').")
        return
    base = _base[(_base["periodo"] == periodo_escolhido) & _base["Valor_Venda"].notna()]
    if base.empty:
        print(f"Sem dados para período {periodo_escolhido}.")
        return
    tab = (base.groupby(["Produto", uf_col])["Valor_Venda"]
                .mean()
                .unstack(uf_col)
                .sort_index())
    fig = plt.figure(figsize=(10, 6))
    ax = fig.gca()
    im = ax.imshow(tab.values, aspect="auto")
    ax.set_xticks(np.arange(tab.shape[1])); ax.set_xticklabels(list(tab.columns), rotation=45, ha="right")
    ax.set_yticks(np.arange(tab.shape[0])); ax.set_yticklabels(list(tab.index))
    ax.set_title(f"Heatmap – preço médio por Produto x UF ({periodo_escolhido})")
    ax.set_xlabel("UF"); ax.set_ylabel("Produto")
    for i in range(tab.shape[0]):
        for j in range(tab.shape[1]):
            val = tab.values[i, j]
            if np.isfinite(val):
                ax.text(j, i, f"R$ {val:,.2f}".replace(",", "X").replace(".", ",").replace("X","."), 
                        ha="center", va="center")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    _savefig(fig, f"item05_heatmap_produto_uf_{periodo_escolhido}.png", title_prefix="Item 5")


# ============ (Item 6) Relações por período (etanol/gasolina, spread diesel-gasolina, gnv/gasolina) ============
def _norm(s: str) -> str:
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii","ignore").decode("ascii")
    return " ".join(s.lower().split())


mix = (_base.assign(mes=_base["Data_Coleta"].dt.to_period("M").dt.to_timestamp())
         .groupby(["mes","periodo","Produto"])["Valor_Venda"]
         .mean()
         .unstack("Produto"))


if not mix.empty:
    cols = list(mix.columns)
    norm_map = {c: _norm(c) for c in cols}
    keys = {
        "gasolina": ["gasolina"],
        "etanol":   ["etanol","alcool"],
        "diesel":   ["diesel","oleo diesel"],
        "gnv":      ["gnv","gas natural veicular","gas natural"]
    }
    def family_columns(fam):
        return [c for c in cols if any(k in norm_map[c] for k in keys[fam])]

    fam_series = {}
    for fam in ["gasolina","etanol","diesel","gnv"]:
        fam_cols = family_columns(fam)
        if fam_cols:
            fam_series[fam] = mix[fam_cols].mean(axis=1, skipna=True)

    out = pd.concat(fam_series, axis=1) if fam_series else pd.DataFrame(index=mix.index)

    if not out.empty:
        out = out.reset_index()  # traz 'mes' e 'periodo'

        if {"etanol","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["etanol"]/s["gasolina"], label=per)
            ax.axhline(0.70, linestyle="--")
            ax.set_title("Razão Etanol/Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Razão")
            ax.legend()
            _savefig(fig, "item06_ratio_etanol_gasolina.png", title_prefix="Item 6")


        if {"diesel","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["diesel"] - s["gasolina"], label=per)
            ax.set_title("Spread Diesel - Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Diferença (R$)")
            ax.legend()
            _savefig(fig, "item06_spread_diesel_gasolina.png", title_prefix="Item 6")

        if {"gnv","gasolina"}.issubset(out.columns):
            fig = plt.figure()
            ax = fig.gca()
            for per in periodos:
                s = out[out["periodo"]==per]
                if not s.empty:
                    ax.plot(s["mes"], s["gnv"]/s["gasolina"], label=per)
            ax.set_title("Razão GNV/Gasolina por período (mensal)")
            ax.set_xlabel("Mês")
            ax.set_ylabel("Razão")
            ax.legend()
            _savefig(fig, "item06_ratio_gnv_gasolina.png", title_prefix="Item 6")
else:
    print("Sem dados para relações entre combustíveis.")


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/45865034.py:56: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  m = (_base.dropna(subset=["Data_Coleta","Produto","Valor_Venda"])


Figura salva: plots/item02_serie_diesel.png
Figura salva: plots/item02_serie_diesel_s10.png
Figura salva: plots/item02_serie_diesel_s50.png
Figura salva: plots/item02_serie_etanol.png
Figura salva: plots/item02_serie_gasolina.png
Figura salva: plots/item02_serie_gasolina_aditivada.png
Figura salva: plots/item02_serie_gnv.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/45865034.py:92: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=periodos, showfliers=True)


Figura salva: plots/item02_boxplot_diesel.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/45865034.py:92: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  ax.boxplot(data, labels=periodos, showfliers=True)


...

Figura salva: plots/item02_boxplot_gnv.png
Figura salva: plots/item03_vol_diesel.png
Figura salva: plots/item03_vol_diesel_s10.png
Figura salva: plots/item03_vol_diesel_s50.png
Figura salva: plots/item03_vol_etanol.png
Figura salva: plots/item03_vol_gasolina.png
Figura salva: plots/item03_vol_gasolina_aditivada.png
Figura salva: plots/item03_vol_gnv.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/45865034.py:119: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  marg = (_base.groupby(["Produto","periodo"])["Margem_%"]


Figura salva: plots/item04_margem_diesel.png
Figura salva: plots/item04_margem_diesel_s10.png
Figura salva: plots/item04_margem_diesel_s50.png
Figura salva: plots/item04_margem_etanol.png
Figura salva: plots/item04_margem_gasolina.png
Figura salva: plots/item04_margem_gasolina_aditivada.png
Figura salva: plots/item04_margem_gnv.png


/var/folders/nf/bf0gd8w92ps1c9zytff6zcfc0000gn/T/ipykernel_4601/45865034.py:175: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mix = (_base.assign(mes=_base["Data_Coleta"].dt.to_period("M").dt.to_timestamp())


Figura salva: plots/item06_ratio_etanol_gasolina.png
Figura salva: plots/item06_spread_diesel_gasolina.png
Figura salva: plots/item06_ratio_gnv_gasolina.png


# 🔧 Complementos automáticos — Questionário completo
*(adicionados em 2025-10-02 02:21)*

> Estas células foram adicionadas para cobrir **todas** as questões do case técnico,
mantendo o código simples e preservando o conteúdo original do seu notebook.
Caso você já tenha respondido a alguma questão acima, estas células não alteram o que foi feito;
apenas centralizam respostas **mínimas e diretas** para cada item, usando a variável `df`.

## Pré-requisito: localizar o DataFrame principal

In [ ]:
# Esta célula tenta localizar automaticamente um DataFrame já carregado.
import pandas as pd
_found = {name: obj for name, obj in globals().items() if isinstance(obj, pd.DataFrame)}
if 'df' not in globals():
    if _found:
        _main_name = next(iter(_found))
        df = _found[_main_name]
        print(f"Usando DataFrame detectado: {_main_name} → 'df'")
    else:
        raise RuntimeError("Nenhum DataFrame foi encontrado nas variáveis ativas. "
                           "Carregue seu dataset e defina-o como 'df' antes de executar os complementos.")
display(df.head(2))

## Parte 1 – Leitura de Dados

### 1. Importação do dataset (já existente no notebook)

In [ ]:
# Pressupõe-se que o dataset já foi importado acima. Apenas confirmamos a existência de `df`.
df.shape

### 2. Contextualização do problema de negócio

In [ ]:
# Ajuste o texto abaixo se desejar personalizar.
contexto = """
Este notebook realiza uma análise exploratória de dados (EDA) em um conjunto que contém
variáveis numéricas e categóricas, com o objetivo de identificar padrões, qualidade de dados,
tendências e possíveis relações que subsidiem decisões de negócio (como segmentação,
previsão de demanda, detecção de anomalias ou priorização de alvos).
"""
print(contexto.strip())

### 3. Análise exploratória de dados (visão geral)

In [ ]:
# Visão geral simples do EDA: tamanho, tipos, amostras e valores ausentes.
print("Formato:", df.shape)
print("\nTipos:\n", df.dtypes)
print("\nValores ausentes (top 10):\n", df.isna().sum().sort_values(ascending=False).head(10))
print("\nAmostra:\n", df.sample(min(5, len(df)), random_state=42))

### 4. Exiba as 4 primeiras linhas

In [ ]:
df.head(4)

### 5. Exiba as 3 últimas linhas

In [ ]:
df.tail(3)

### 6. Descreva as principais variáveis

In [ ]:
# Listamos colunas numéricas e categóricas; ajuste os comentários conforme o seu contexto.
num_cols = df.select_dtypes(include='number').columns.tolist()
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
print("Numéricas:", num_cols[:10], "..." if len(num_cols)>10 else "")
print("Categóricas:", cat_cols[:10], "..." if len(cat_cols)>10 else "")

### 7. Formato, tipos, ausentes e duplicações

In [ ]:
print("Formato:", df.shape)
print("\nTipos:\n", df.dtypes)
print("\nValores ausentes por coluna:\n", df.isna().sum())
print("\nDuplicações:", df.duplicated().sum())

## Parte 2 – Estruturas de Dados e Operações em Python

### 1. Duas listas: numéricas e categóricas

In [ ]:
lista_numericas = df.select_dtypes(include='number').columns.tolist()
lista_categoricas = df.select_dtypes(exclude='number').columns.tolist()
lista_numericas, lista_categoricas

### 2. Dicionário coluna → tipo ('numérica'|'categórica')

In [ ]:
tipo_cols = {c: ('numérica' if pd.api.types.is_numeric_dtype(df[c]) else 'categórica') for c in df.columns}
tipo_cols

### 3. Tupla com nomes das colunas

In [ ]:
tupla_colunas = tuple(df.columns); tupla_colunas[:10], len(tupla_colunas)

### 4. Tupla com números (soma, maior, menor)

In [ ]:
# Escolhe automaticamente a primeira coluna numérica disponível
import numpy as np
num_cols = df.select_dtypes(include='number').columns.tolist()
if not num_cols:
    raise RuntimeError("Não há colunas numéricas para esta questão.")
col_exemplo = num_cols[0]
tupla_numeros = tuple(df[col_exemplo].dropna().values)
soma = sum(tupla_numeros)
maior = max(tupla_numeros) if tupla_numeros else None
menor = min(tupla_numeros) if tupla_numeros else None
col_exemplo, soma, maior, menor

### 5. Conjunto (set) com valores únicos de uma categórica

In [ ]:
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
if not cat_cols:
    raise RuntimeError("Não há colunas categóricas para esta questão.")
cat_exemplo = cat_cols[0]
conjunto_unicos = set(df[cat_exemplo].dropna().astype(str))
cat_exemplo, list(sorted(conjunto_unicos))[:20]

### 6. Seleções no DataFrame

In [ ]:
# a) Selecione uma coluna usando indexação
col0 = df.columns[0]
serie_col0 = df[col0]

# b) Selecione as primeiras 5 linhas com slicing
primeiras_5 = df[:5]
col0, serie_col0.head(3), primeiras_5

## Parte 3 – NumPy

### 1. Extraia colunas numéricas em um array NumPy

In [ ]:
import numpy as np
X_num = df.select_dtypes(include='number').to_numpy()
X_num[:3], X_num.shape

### 2. Converta um array NumPy de volta para DataFrame mantendo nomes

In [ ]:
cols_num = df.select_dtypes(include='number').columns
df_num_copia = pd.DataFrame(X_num, columns=cols_num)
df_num_copia.head(3)

### 3. Extraia uma matriz NumPy com as colunas numéricas

In [ ]:
matriz_numerica = df.select_dtypes(include='number').values
type(matriz_numerica), matriz_numerica.shape

### 4. Aplique reshape() em um subconjunto

In [ ]:
# Exemplo simples: pega a primeira coluna numérica e cria um vetor coluna (n, 1)
col_ex = df.select_dtypes(include='number').columns
if len(col_ex)==0:
    raise RuntimeError("Sem colunas numéricas para reshape.")
arr = df[col_ex[0]].dropna().to_numpy()
arr2d = arr.reshape(-1, 1)
arr.shape, arr2d.shape

### 5. Média, mediana e desvio padrão (NumPy)

In [ ]:
cols = df.select_dtypes(include='number').columns
resumo = {}
for c in cols:
    a = df[c].dropna().to_numpy()
    if a.size:
        resumo[c] = {
            "media": float(np.mean(a)),
            "mediana": float(np.median(a)),
            "desvio_padrao": float(np.std(a, ddof=1)) if a.size>1 else 0.0
        }
resumo

## Parte 4 – Pandas

### 1. Seleção de linhas por condição

In [ ]:
# Condição simples: se houver alguma coluna numérica, filtra valores acima da média na primeira coluna
num_cols = df.select_dtypes(include='number').columns.tolist()
if not num_cols:
    raise RuntimeError("Sem colunas numéricas para filtrar.")
c = num_cols[0]
filtro = df[c] > df[c].mean()
df_filtrado = df[filtro]
c, df[c].mean(), df_filtrado.head(3)

### 2. Groupby por categórica e médias de duas numéricas

In [ ]:
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_cols = df.select_dtypes(include='number').columns.tolist()
if len(cat_cols)==0 or len(num_cols)<2:
    raise RuntimeError("Requer pelo menos 1 categórica e 2 numéricas para o groupby.")
gcol = cat_cols[0]
gdf = df.groupby(gcol)[num_cols[:2]].mean(numeric_only=True).reset_index()
gcol, num_cols[:2], gdf.head(3)

### 3. merge() com dados agregados por categoria

In [ ]:
agg = df.groupby(gcol)[num_cols[:2]].mean(numeric_only=True).rename(
    columns={num_cols[0]: f"{num_cols[0]}_mean", num_cols[1]: f"{num_cols[1]}_mean"}
).reset_index()
df_merged = pd.merge(df, agg, on=gcol, how='left')
df_merged.head(3)

### 4. Resumo estatístico (.describe) numéricas

In [ ]:
df.select_dtypes(include='number').describe()

### 5. Resumo de categóricas com crosstab()

In [ ]:
# Crosstab simples entre duas primeiras categóricas (se existirem)
cat_cols = df.select_dtypes(exclude='number').columns.tolist()
if len(cat_cols) >= 2:
    ct = pd.crosstab(df[cat_cols[0]], df[cat_cols[1]])
    ct.head()
else:
    print("Menos de duas colunas categóricas disponíveis para crosstab.")

## Parte 5 – Visualização com Matplotlib e Seaborn

### 1. Gráfico de linha

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_cols = df.select_dtypes(include='number').columns.tolist()
if len(num_cols)==0:
    raise RuntimeError("Sem colunas numéricas para o gráfico de linha.")
y = df[num_cols[0]].dropna().reset_index(drop=True)
plt.figure()
plt.plot(y.reset_index(drop=True))
plt.title(f"Gráfico de linha — {num_cols[0]}")
plt.xlabel("Índice")
plt.ylabel(num_cols[0])
plt.show()

### 2. Gráfico de barras da contagem de categorias

In [ ]:
import matplotlib.pyplot as plt

cat_cols = df.select_dtypes(exclude='number').columns.tolist()
if len(cat_cols)==0:
    raise RuntimeError("Sem colunas categóricas para o gráfico de barras.")
vc = df[cat_cols[0]].astype(str).value_counts().head(15)
plt.figure()
vc.plot(kind='bar')
plt.title(f"Contagem — {cat_cols[0]} (top 15)")
plt.xlabel(cat_cols[0])
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()

### 3. Histograma de uma variável numérica

In [ ]:
import matplotlib.pyplot as plt

num_cols = df.select_dtypes(include='number').columns.tolist()
if len(num_cols)==0:
    raise RuntimeError("Sem colunas numéricas para histograma.")
plt.figure()
df[num_cols[0]].dropna().plot(kind='hist', bins=30)
plt.title(f"Histograma — {num_cols[0]}")
plt.xlabel(num_cols[0])
plt.ylabel("Contagem")
plt.show()

### 4. Boxplot por categórica

In [ ]:
import matplotlib.pyplot as plt

cat_cols = df.select_dtypes(exclude='number').columns.tolist()
num_cols = df.select_dtypes(include='number').columns.tolist()
if len(cat_cols)==0 or len(num_cols)==0:
    raise RuntimeError("Requer pelo menos 1 categórica e 1 numérica para boxplot.")
plt.figure()
df.boxplot(column=num_cols[0], by=cat_cols[0], rot=45)
plt.title(f"Boxplot de {num_cols[0]} por {cat_cols[0]}")
plt.suptitle("")
plt.xlabel(cat_cols[0])
plt.ylabel(num_cols[0])
plt.tight_layout()
plt.show()

### 5. Mapa de calor da correlação (numéricas)

In [ ]:
import numpy as np, matplotlib.pyplot as plt

num_df = df.select_dtypes(include='number')
if num_df.shape[1] == 0:
    raise RuntimeError("Sem colunas numéricas para correlação.")
corr = num_df.corr(numeric_only=True)
plt.figure()
plt.imshow(corr, cmap='viridis', interpolation='nearest')
plt.title("Matriz de correlação (numéricas)")
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.tight_layout()
plt.show()

### 6. Pairplot (seaborn)

In [ ]:
# Pairplot simples das 4 primeiras colunas numéricas para evitar excesso de figuras
import seaborn as sns
num_cols = df.select_dtypes(include='number').columns.tolist()
if len(num_cols) >= 2:
    sns.pairplot(df[num_cols[:4]].dropna())
else:
    print("Colunas numéricas insuficientes para pairplot.")

## Parte 6 – Exportando DataFrames em CSV

### 1–3. Exports: 'dados_trat.csv' e 'subset_numericas.csv'

In [ ]:
# 1 e 2: salvar df completo SEM índice
df.to_csv("dados_trat.csv", index=False)
print("Arquivo salvo:", "dados_trat.csv")

# 3: salvar apenas colunas numéricas
df.select_dtypes(include='number').to_csv("subset_numericas.csv", index=False)
print("Arquivo salvo:", "subset_numericas.csv")